In [1]:
from __future__ import print_function
# import keras
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import SeparableConv2D, MaxPooling2D
from tensorflow.keras import backend as K
import time
from PIL import Image as pil_image
from tensorflow.keras.preprocessing.image import save_img
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization



start_time = time.time()

batch_size = 128
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()



x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1,img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1,img_rows, img_cols)
    input_img = (1,img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_img = (img_rows, img_cols, 1)


# x_train = x_train.reshape(x_train.shape[-1], 3)
# channels = input_img[0].shape[-1]
x_test.shape

x_train shape: (60000, 28, 28)
60000 train samples
10000 test samples


(10000, 28, 28, 1)

In [2]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''



def model1(weights_path=None):

    model = Sequential()
    model.add(SeparableConv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_img , padding = 'same'))
    model.add(BatchNormalization())
    model.add(SeparableConv2D(64, (3, 3), activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    
    model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test)) 
    score = model.evaluate(x_test, y_test, verbose=0)
    model.save_weights('my_model_weights.h5')
    model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
    model.pop()
    print(len(model.layers))
    return score


score = model1()

print('Test loss:', score[0])
print('Test accuracy:', score[1])

print("--- %s seconds ---" % (time.time() - start_time))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d (SeparableC (None, 28, 28, 32)        73        
_________________________________________________________________
batch_normalization_v1 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 28, 28, 64)        2400      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)    

In [3]:
import numpy as np

# model = model(include_top = False, inputs=mnist)
np.shape(x_train)

(60000, 28, 28, 1)

In [4]:
def normalize(w):
    """utility function to normalize a tensor.
    # Arguments
                x_train: An input tensor.
    # Returns
        The normalized input tensor.
    """
    return w / (K.sqrt(K.mean(K.square(w))) + K.epsilon())


In [5]:
print(len(x_train))
np.shape(x_train)

60000


(60000, 28, 28, 1)

In [6]:
def deprocess_image(y):
    """utility function to convert a float array into a valid uint8 image.
    # Arguments
                y: A numpy-array representing the generated image.
    # Returns
        A processed numpy-array, which could be used in e.g. imshow.
    """
    # normalize tensor: center on 0., ensure std is 0.25
    y -= np.mean(y)
    y /= (np.std(y) + K.epsilon())
    y *= 0.25

    # clip to [0, 1]
    y += 0.5
    y = np.clip(y, 0, 1)
    y*=255
    # convert to RGB array
#     x_train *= 255
    if K.image_data_format() == 'channels_first':
        y = x.transpose((1, 2, 0))
    y = np.clip(y, 0, 255).astype('uint8')
    return y

In [7]:
def process_image(o, former):
    """utility function to convert a valid uint8 image back into a float array.
       Reverses `deprocess_image`.
    # Arguments
        x_train: A numpy-array, which could be used in e.g. imshow.
        former: The former numpy-array.
                Need to determine the former mean and variance.
    # Returns
        A processed numpy-array representing the generated image.
    """
    if K.image_data_format() == 'channels_first':
        o = o.transpose((2, 0, 1))
    return (o / 255 - 0.5) * 4 * former.std() + former.mean()


In [8]:
def visualize_layer(model,
                    layer_name,
                    step=1.,
                    epochs=1,
                    upscaling_steps=9,
                    upscaling_factor=1.2,
                    output_dim=(412, 412),
                    filter_range=(0, None)):
    """Visualizes the most relevant filters of one conv-layer in a certain model.
    # Arguments
        model: The model containing layer_name.
        layer_name: The name of the layer to be visualized.
                    Has to be a part of model.
        step: step size for gradient ascent.
        epochs: Number of iterations for gradient ascent.
        upscaling_steps: Number of upscaling steps.
                         Starting image is in this case (80, 80).
        upscaling_factor: Factor to which to slowly upgrade
                          the image towards output_dim.
        output_dim: [img_width, img_height] The output image dimensions.
        filter_range: Tupel[lower, upper]
                      Determines the to be computed filter numbers.
                      If the second value is `None`,
                      the last filter will be inferred as the upper boundary.
    """

    def _generate_filter_image(input_img,
                               layer_output,
                               filter_index):
        """Generates image for one particular filter.
        # Arguments
            input_img: The input-image Tensor.
            layer_output: The output-image Tensor.
            filter_index: The to be processed filter number.
                          Assumed to be valid.
        #Returns
            Either None if no image could be generated.
            or a tuple of the image (array) itself and the last loss.
        """
        s_time = time.time() # Start and record the time for each epoch

        # we build a loss function that maximizes the activation
        # of the nth filter of the layer considered
        if K.image_data_format() == 'channels_first':
            loss = K.mean(layer_output[:, filter_index, :, :])
        else:
            loss = K.mean(layer_output[:, :, :, filter_index])
        num_channels = input_img[0].shape[-1] # --------------------------------------------------------------
        print(num_channels)
        # we compute the gradient of the input picture wrt this loss
        grads = K.gradients(loss, input_img)[0]

        # normalization trick: we normalize the gradient
        grads = normalize(grads)

        # this function returns the loss and grads given the input picture
        iterate = K.function([input_img], [loss, grads])

        # we start from a gray image with some random noise
        intermediate_dim = tuple(
            int(x_train / (upscaling_factor ** upscaling_steps)) for x_train in output_dim)
        if K.image_data_format() == 'channels_first':
            input_img_data = np.random.random(
                (1, 1, intermediate_dim[0], intermediate_dim[1])) # change to 3 for colour images
        else:
            input_img_data = np.random.random(
                (1, intermediate_dim[0], intermediate_dim[1], 1)) # change to 3 for colour images
        input_img_data = (input_img_data - 0.5) * 20 + 128

        # Slowly upscaling towards the original size prevents
        # a dominating high-frequency of the to visualized structure
        # as it would occur if we directly compute the 412d-image.
        # Behaves as a better starting point for each following dimension
        # and therefore avoids poor local minima
        for up in reversed(range(upscaling_steps)):
            # we run gradient ascent for e.g. 20 steps
            for _ in range(epochs):
                print(type(input_img_data))
                loss_value, grads_value = iterate([input_img_data])
                input_img_data += grads_value * step

                # some filters get stuck to 0, we can skip them
                if loss_value <= K.epsilon():
                    return None

            # Calulate upscaled dimension
            intermediate_dim = tuple(
                int(x_train / (upscaling_factor ** up)) for x_train in output_dim)
            # Upscale
            img = deprocess_image(input_img_data[0])
            print(np.shape(img))
            img = np.reshape(img, [img.shape[0], img.shape[1]])
            img = np.array(pil_image.fromarray(img).resize(intermediate_dim,
                                                           pil_image.BICUBIC))
            input_img_data = process_image(img, input_img_data[0]).reshape([1,img.shape[0],img.shape[1],1])
#             input_img_data = [process_image(img, input_img_data[0])] # Changing this to make it compatible with grey scale imgs

        # decode the resulting input image
        img = deprocess_image(input_img_data[0])
        e_time = time.time()
        print('Costs of filter {:3}: {:5.0f} ( {:4.2f}s )'.format(filter_index,
                                                                  loss_value,
                                                                  e_time - s_time))
        return img, loss_value

    def _draw_filters(filters, n=None):
        """Draw the best filters in a nxn grid.
        # Arguments
            filters: A List of generated images and their corresponding losses
                     for each processed filter.
            n: dimension of the grid.
               If none, the largest possible square will be used
        """
        if n is None:
            n = int(np.floor(np.sqrt(len(filters))))

        # the filters that have the highest loss are assumed to be better-looking.
        # we will only keep the top n*n filters.
        filters.sort(key=lambda x_train: x_train[1], reverse=True)
        filters = filters[:n * n]

        # build a black picture with enough space for
        # e.g. our 8 x 8 filters of size 412 x 412, with a 5px margin in between
        MARGIN = 5
        width = n * output_dim[0] + (n - 1) * MARGIN
        height = n * output_dim[1] + (n - 1) * MARGIN
        stitched_filters = np.zeros((width, height, 3), dtype='uint8')

        # fill the picture with our saved filters
        for i in range(n):
            for j in range(n):
                img, _ = filters[i * n + j]
                width_margin = (output_dim[0] + MARGIN) * i
                height_margin = (output_dim[1] + MARGIN) * j
                stitched_filters[
                    width_margin: width_margin + output_dim[0],
                    height_margin: height_margin + output_dim[1], :] = img

        # save the result to disk
        save_img('vgg_{0:}_{1:}x{1:}.png'.format(layer_name, n), stitched_filters)

    # this is the placeholder for the input images
    assert len(model.inputs) == 1
    input_img = model.inputs # changed from model.inputs[0]

    # get the symbolic outputs of each "key" layer (we gave them unique names).
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])

    output_layer = layer_dict[layer_name]
    assert isinstance(output_layer, layers.SeparableConv2D)

    # Compute to be processed filter range
    filter_lower = filter_range[0]
    filter_upper = (filter_range[1]
                    if filter_range[1] is not None
                    else (output_layer.get_weights()[1])).shape[-1]
    assert(filter_lower >= 0
           and filter_upper <= output_layer.get_weights()[1].shape[-1]
           and filter_upper > filter_lower)
    print('Compute filters {:} to {:}'.format(filter_lower, filter_upper))

    # iterate through each filter and generate its corresponding image
    processed_filters = []
    for f in range(filter_lower, filter_upper):
        img_loss = _generate_filter_image(input_img, output_layer.output, f)

        if img_loss is not None:
            processed_filters.append(img_loss)

    print('{} filter processed.'.format(len(processed_filters)))
    # Finally draw and store the best filters to disk
    _draw_filters(processed_filters)


if __name__ == '__main__':
    # the name of the layer we want to visualize
    # (see model definition at keras/applications/vgg16.py)
    LAYER_NAME = 'separable_conv2d_1'

    # build the VGG16 network with ImageNet weights
    vgg = load_model('my_model.h5')
    print('Model loaded.')
    vgg.summary()

    # example function call
    visualize_layer(vgg, LAYER_NAME)

Model loaded.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d (SeparableC (None, 28, 28, 32)        73        
_________________________________________________________________
batch_normalization_v1 (Batc (None, 28, 28, 32)        128       
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 28, 28, 64)        2400      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               1605760

(238, 238, 1)
<class 'numpy.ndarray'>
(286, 286, 1)
<class 'numpy.ndarray'>
(343, 343, 1)
Costs of filter  17:     0 ( 3.00s )
1
<class 'numpy.ndarray'>
(79, 79, 1)
<class 'numpy.ndarray'>
(95, 95, 1)
<class 'numpy.ndarray'>
(114, 114, 1)
<class 'numpy.ndarray'>
(137, 137, 1)
<class 'numpy.ndarray'>
(165, 165, 1)
<class 'numpy.ndarray'>
(198, 198, 1)
<class 'numpy.ndarray'>
(238, 238, 1)
<class 'numpy.ndarray'>
(286, 286, 1)
<class 'numpy.ndarray'>
(343, 343, 1)
Costs of filter  18:     0 ( 2.83s )
1
<class 'numpy.ndarray'>
(79, 79, 1)
<class 'numpy.ndarray'>
(95, 95, 1)
<class 'numpy.ndarray'>
(114, 114, 1)
<class 'numpy.ndarray'>
(137, 137, 1)
<class 'numpy.ndarray'>
(165, 165, 1)
<class 'numpy.ndarray'>
(198, 198, 1)
<class 'numpy.ndarray'>
(238, 238, 1)
<class 'numpy.ndarray'>
(286, 286, 1)
<class 'numpy.ndarray'>
(343, 343, 1)
Costs of filter  19:     0 ( 3.85s )
1
<class 'numpy.ndarray'>
(79, 79, 1)
<class 'numpy.ndarray'>
(95, 95, 1)
<class 'numpy.ndarray'>
(114, 114, 1)
<class 

(198, 198, 1)
<class 'numpy.ndarray'>
(238, 238, 1)
<class 'numpy.ndarray'>
(286, 286, 1)
<class 'numpy.ndarray'>
(343, 343, 1)
Costs of filter  39:     0 ( 2.57s )
1
<class 'numpy.ndarray'>
(79, 79, 1)
<class 'numpy.ndarray'>
(95, 95, 1)
<class 'numpy.ndarray'>
(114, 114, 1)
<class 'numpy.ndarray'>
(137, 137, 1)
<class 'numpy.ndarray'>
(165, 165, 1)
<class 'numpy.ndarray'>
(198, 198, 1)
<class 'numpy.ndarray'>
(238, 238, 1)
<class 'numpy.ndarray'>
(286, 286, 1)
<class 'numpy.ndarray'>
(343, 343, 1)
Costs of filter  40:     0 ( 2.70s )
1
<class 'numpy.ndarray'>
(79, 79, 1)
<class 'numpy.ndarray'>
(95, 95, 1)
<class 'numpy.ndarray'>
(114, 114, 1)
<class 'numpy.ndarray'>
(137, 137, 1)
<class 'numpy.ndarray'>
(165, 165, 1)
<class 'numpy.ndarray'>
(198, 198, 1)
<class 'numpy.ndarray'>
(238, 238, 1)
<class 'numpy.ndarray'>
(286, 286, 1)
<class 'numpy.ndarray'>
(343, 343, 1)
Costs of filter  41:     0 ( 2.72s )
1
<class 'numpy.ndarray'>
(79, 79, 1)
<class 'numpy.ndarray'>
(95, 95, 1)
<class 

(137, 137, 1)
<class 'numpy.ndarray'>
(165, 165, 1)
<class 'numpy.ndarray'>
(198, 198, 1)
<class 'numpy.ndarray'>
(238, 238, 1)
<class 'numpy.ndarray'>
(286, 286, 1)
<class 'numpy.ndarray'>
(343, 343, 1)
Costs of filter  61:     0 ( 2.84s )
1
<class 'numpy.ndarray'>
(79, 79, 1)
<class 'numpy.ndarray'>
(95, 95, 1)
<class 'numpy.ndarray'>
(114, 114, 1)
<class 'numpy.ndarray'>
(137, 137, 1)
<class 'numpy.ndarray'>
(165, 165, 1)
<class 'numpy.ndarray'>
(198, 198, 1)
<class 'numpy.ndarray'>
(238, 238, 1)
<class 'numpy.ndarray'>
(286, 286, 1)
<class 'numpy.ndarray'>
(343, 343, 1)
Costs of filter  62:     0 ( 2.87s )
1
<class 'numpy.ndarray'>
(79, 79, 1)
<class 'numpy.ndarray'>
(95, 95, 1)
<class 'numpy.ndarray'>
(114, 114, 1)
<class 'numpy.ndarray'>
(137, 137, 1)
<class 'numpy.ndarray'>
(165, 165, 1)
<class 'numpy.ndarray'>
(198, 198, 1)
<class 'numpy.ndarray'>
(238, 238, 1)
<class 'numpy.ndarray'>
(286, 286, 1)
<class 'numpy.ndarray'>
(343, 343, 1)
Costs of filter  63:     0 ( 2.77s )
64 fi